# Community analysis

Lukas Krodinger

In [5]:
import community
import networkx as nx
from networkx.algorithms.community import k_clique_communities

In [6]:
# filename of graph to read in
input_filename = "workshop8.graphml"
output_filename = "workshop8-mod.graphml"

Utility function to convert the output of python-Louvain, a dictionary (node -> community id), to a list of sets (community id -> [nodes]).

In [7]:
def dictToSetFormat(dComms, maxCommNum):
    """
    Converts dictionary based community output (node -> community id) to list of sets (communities)
    @param dComms: dictionary based community representation
    @param maxCommNum: maximum number of communities
    @return: list of communities representation
    @author Jeff
    """

    # initialise
    lGroundTruth = [set() for x in range(maxCommNum)]
    # convert each (node : community id) pair to the required set format
    for (name, clusId) in dComms.items():
        lGroundTruth[clusId].add(name)

    return lGroundTruth

In [8]:
# read the reply graph in
graph = nx.readwrite.read_graphml(input_filename)

# clique size
k = 3

# Compute the communities
cpm_comms = list(k_clique_communities(graph.to_undirected(), k))
louvain_comms = community.best_partition(graph.to_undirected())

# converts output of Louvain to the same format as CPM (makes it easier to have one consistent interface)
louvain_comms_amount = max([y for (x, y) in louvain_comms.items()]) + 1
louvain_comms = dictToSetFormat(louvain_comms, louvain_comms_amount)

# Add cpm community ids to nodes
for comm_id, community in enumerate(cpm_comms):
    for member in community:
        graph.nodes[member]["CPM"] = comm_id

# Add louvain community ids to nodes
for comm_id, community in enumerate(louvain_comms):
    for member in community:
        graph.nodes[member]["Louvain"] = comm_id

# Output modified graph with prefix mod2 to original file.
nx.readwrite.write_graphml(graph, output_filename, infer_numeric_types=True)